In [1]:
using Images,CuArrays,Flux
using Flux:@treelike, Tracker
using Base.Iterators: partition
using Random
using Statistics

# TODO : Create Image Pool
#        Write Identity Loss

"""
utils.jl
"""
function load_image(filename)
    img = load(filename)
    img = Float32.(channelview(img))
end

function load_dataset(path,imsize)
   imgs = []
   for r in readdir(path)
        img_path = string(path,r)
        push!(imgs,load_image(img_path))
    end
    reshape(hcat(imgs...),imsize,imsize,3,length(imgs))
end

function make_minibatch(X, idxs,size=256)
    """
    size : Image dimension
    """
    X_batch = Array{Float32}(undef, size, size, 3, length(idxs))
    for i in 1:length(idxs)
        X_batch[:,:,:, i] = Float32.(X[:,:,:,idxs[i]])
    end
    return X_batch
end

function nullify_grad!(p)
  if typeof(p) <: TrackedArray
    p.grad .= 0.0f0
  end
  return p
end

function zero_grad!(model)
  model = mapleaves(nullify_grad!, model)
end

function norm(x)
    2.0 .* x .- 1.0
end

function denorm(x)
   (x .+ 1.0)./(2.0) 
end

expand_dims(x,n::Int) = reshape(x,ones(Int64,n)...,size(x)...)
function squeeze(x) 
    if size(x)[end] != 1
        return dropdims(x, dims = tuple(findall(size(x) .== 1)...))
    else
        # BATCH_SIZE = 1
        int_val = dropdims(x, dims = tuple(findall(size(x) .== 1)...))
        return reshape(int_val,size(int_val)...,1)
    end
end

drop_first_two(x) = dropdims(x,dims=(1,2))

# BatchNorm Wrapper
function BatchNormWrap(out_ch)
    Chain(x->expand_dims(x,2),
    BatchNorm(out_ch),
    x->squeeze(x))
end

┌ Warning: Flux is only supported with CuArrays v0.9.
│ Try running `] pin CuArrays@0.9`.
└ @ Flux.CUDA /home/shreyas/.julia/packages/Flux/WSB7k/src/cuda/cuda.jl:12


BatchNormWrap (generic function with 1 method)

In [6]:
dataA = load_dataset("../data/trainA/",256)[1:100,:,:,:] |> gpu
dataB = load_dataset("../data/trainB/",256)[1:100,:,:,:] |> gpu

100×3×256×256 CuArray{Float32,4}:
[:, :, 1, 1] =
 1.0       0.0235294  0.160784
 1.0       0.278431   0.223529
 1.0       0.156863   0.454902
 0.988235  0.0431373  0.164706
 0.988235  0.341176   0.262745
 0.988235  0.184314   0.45098 
 0.988235  0.0823529  0.156863
 0.988235  0.364706   0.235294
 0.988235  0.196078   0.462745
 1.0       0.0588235  0.14902 
 1.0       0.329412   0.247059
 1.0       0.152941   0.470588
 0.988235  0.0705882  0.145098
 ⋮                            
 0.992157  0.152941   0.705882
 0.992157  0.0862745  0.627451
 1.0       0.054902   0.666667
 1.0       0.152941   0.764706
 1.0       0.0862745  0.584314
 1.0       0.054902   0.584314
 1.0       0.145098   0.788235
 1.0       0.0784314  0.517647
 1.0       0.0470588  0.490196
 1.0       0.133333   0.690196
 1.0       0.0745098  0.509804
 1.0       0.0470588  0.435294

[:, :, 2, 1] =
 0.45098   0.0         0.54902 
 0.25098   0.87451     0.54902 
 0.137255  0.298039    0.729412
 0.439216  0.0         0.533333
 

In [2]:
"""
generator.jl
"""

UNetConvBlock(in_chs, out_chs, kernel = (3, 3)) =
    Chain(Conv(kernel, in_chs=>out_chs,pad = (1, 1)) ,BatchNormWrap(out_chs)...,x->leakyrelu.(x),
          Conv(kernel, out_chs=>out_chs,pad = (1, 1)),BatchNormWrap(out_chs)...,x->leakyrelu.(x))

struct UNetUpBlock
    upsample
    conv_layer
end

@treelike UNetUpBlock

UNetUpBlock(in_chs::Int, out_chs::Int, kernel = (3, 3)) =
    UNetUpBlock(ConvTranspose((2, 2), in_chs=>out_chs, stride=(2, 2)),
                Chain(Conv(kernel, in_chs=>out_chs,pad = (1, 1)),BatchNormWrap(out_chs)...,x->leakyrelu.(x),
                Conv(kernel, out_chs=>out_chs,pad = (1, 1)),BatchNormWrap(out_chs)...,x->leakyrelu.(x)))

function (u::UNetUpBlock)(x, bridge)
    x = u.upsample(x)
    u.conv_layer(cat(x, bridge, dims = 3))
end

struct UNet
    pool_layer
    conv_blocks
    up_blocks
end

@treelike UNet

# This is to be used for Background and Foreground segmentation
function UNet()
    pool_layer = MaxPool((2, 2))
    conv_blocks = (UNetConvBlock(3, 64), UNetConvBlock(64, 128), UNetConvBlock(128, 256),
                   UNetConvBlock(256, 512), UNetConvBlock(512, 1024))
    up_blocks = (UNetUpBlock(1024, 512), UNetUpBlock(512, 256), UNetUpBlock(256, 128),
                 UNetUpBlock(128, 64), Conv((1, 1), 64=>3))
    UNet(pool_layer, conv_blocks, up_blocks)
end

function (u::UNet)(x)
    outputs = Vector(undef, 5)
    outputs[1] = u.conv_blocks[1](x)
    for i in 2:5
        pool_x = u.pool_layer(outputs[i - 1])
        outputs[i] = u.conv_blocks[i](pool_x)
    end
    up_x = outputs[end]
    for i in 1:4
        up_x = u.up_blocks[i](up_x, outputs[end - i])
    end
    tanh.(u.up_blocks[end](up_x))
end

In [3]:
out = UNet()(ones(256,256,3,2))

A
A
A
A


Tracked 256×256×3×2 Array{Float32,4}:
[:, :, 1, 1] =
  0.999534  -0.236976    0.984831   …   0.999209  -0.274376   -0.948484 
  0.904897  -0.367145   -0.974328       0.560452   0.680494   -0.954878 
 -0.96968   -0.992345   -0.952657      -0.565723  -0.0415492  -0.885268 
 -0.998865  -0.989182   -0.954232      -0.370259   0.578097   -0.966648 
 -0.618327   0.988929   -0.485399      -0.305766  -0.947536   -0.547377 
 -0.989433  -0.0152619  -0.803056   …   0.998531   0.749391   -0.460169 
 -0.906661   0.913384   -0.0958578     -0.528856   0.213911   -0.800023 
 -0.999368  -0.660914   -0.891079       0.155959   0.443433   -0.915801 
 -0.502572   0.6753     -0.547302       0.190943   0.267268    0.356757 
 -0.991812   0.11329    -0.652053       0.999515  -0.669832    0.0746427
 -0.859383   0.807388   -0.939246   …   0.395367  -0.949545   -0.668462 
 -0.999891   0.264103   -0.938966       0.984819   0.94457    -0.843073 
 -0.70824   -0.531768   -0.485194       0.408948   0.681968   -0.771218

In [3]:
"""
discriminator.jl
"""
ConvBlock(in_ch::Int,out_ch::Int) = 
    Chain(Conv((4,4), in_ch=>out_ch,pad = (1, 1), stride=(2,2)),
          BatchNormWrap(out_ch)...,
          x->leakyrelu.(x))

function Discriminator()
    model = Chain(Conv((4,4), 3=>8,pad = (1, 1), stride=(2,2)),x->leakyrelu.(x)) 
    model = Chain(model...,ConvBlock(8,16)...)
    model = Chain(model...,ConvBlock(16,32)...)
    model = Chain(model...,ConvBlock(32,64)...)
    model = Chain(model...,ConvBlock(64,32)...)
    model = Chain(model...,ConvBlock(32,16)...)
    model = Chain(model...,ConvBlock(16,8)...)
    model = Chain(model...,Chain(Conv((4,4), 8=>1,pad = (1, 1), stride=(2,2))))
    # Note : No sigmoid on last layer of Discriminator
    #        LSGAN can handle it with basic backpropagation
    model
end

Discriminator (generic function with 1 method)

In [26]:
model = Discriminator() |> gpu
drop_first_two(model(ones(256,256,3,2) |> gpu)

Chain(Conv((4, 4), 3=>8), getfield(Main, Symbol("##31#33"))(), Conv((4, 4), 8=>16), getfield(Main, Symbol("##25#27"))(), BatchNorm(16), getfield(Main, Symbol("##26#28"))(), getfield(Main, Symbol("##29#30"))(), Conv((4, 4), 16=>32), getfield(Main, Symbol("##25#27"))(), BatchNorm(32), getfield(Main, Symbol("##26#28"))(), getfield(Main, Symbol("##29#30"))(), Conv((4, 4), 32=>64), getfield(Main, Symbol("##25#27"))(), BatchNorm(64), getfield(Main, Symbol("##26#28"))(), getfield(Main, Symbol("##29#30"))(), Conv((4, 4), 64=>32), getfield(Main, Symbol("##25#27"))(), BatchNorm(32), getfield(Main, Symbol("##26#28"))(), getfield(Main, Symbol("##29#30"))(), Conv((4, 4), 32=>16), getfield(Main, Symbol("##25#27"))(), BatchNorm(16), getfield(Main, Symbol("##26#28"))(), getfield(Main, Symbol("##29#30"))(), Conv((4, 4), 16=>8), getfield(Main, Symbol("##25#27"))(), BatchNorm(8), getfield(Main, Symbol("##26#28"))(), getfield(Main, Symbol("##29#30"))(), Chain(Conv((4, 4), 8=>1), getfield(Main, Symbol("##3

In [5]:
"""
train.jl
"""
# Hyperparameters
NUM_EPOCHS = 100
BATCH_SIZE = 1
dis_lr = 0.0001f0
gen_lr = 0.0001f0
λ₁ = 10.0 # Cycle loss weight for dommain A
λ₂ = 10.0 # Cycle loss weight for domain B
NUM_EXAMPLES = 2 # Temporary for experimentation
VERBOSE_FREQUENCY = 2 # Verbose output after every 2 epochs

# Data Loading
dataA = load_dataset("../data/trainA/",256)[:,:,:,1:NUM_EXAMPLES] |> gpu
dataB = load_dataset("../data/trainB/",256)[:,:,:,1:NUM_EXAMPLES] |> gpu
mb_idxs = partition(shuffle!(collect(1:size(dataA)[end])), BATCH_SIZE)
train_A = [make_minibatch(dataA, i) for i in mb_idxs]
train_B = [make_minibatch(dataB, i) for i in mb_idxs]
println("Loaded Data")

# Define Optimizers
opt_gen = ADAM(gen_lr,(0.5,0.999))
opt_disc_A = ADAM(dis_lr,(0.5,0.999))
opt_disc_B = ADAM(dis_lr,(0.5,0.999))

# Define models
gen_A = UNet() |> gpu # Generator For A->B
gen_B = UNet() |> gpu # Generator For B->A
dis_A = Discriminator() |> gpu # Discriminator For Domain A
dis_B = Discriminator() |> gpu # Discriminator For Domain B
println("Loaded Models")


# Forward prop, backprop, optimise!
function train_step(X_A,X_B) 
    # LABELS #
    real_labels = ones(1,BATCH_SIZE)
    fake_labels = ones(0,BATCH_SIZE)
    
    ### Forward Propagation ###
    zero_grad!(gen_A)
    zero_grad!(gen_B)
    
    println("1")
    fake_B = gen_A(X_A) # Fake image generated in domain B
    fake_B_prob = drop_first_two(dis_B(fake_B)) # Probability that generated image in domain B is real
    real_B_prob = drop_first_two(dis_B(X_B)) # Probability that original image in domain B is real
    println("2")
    
    fake_A = gen_B(X_B) # Fake image generated in domain A
    fake_A_prob = drop_first_two(dis_A(fake_A)) # Probability that generated image in domain A is real
    real_A_prob = drop_first_two(dis_A(X_A)) # Probability that original image in domain A is real
    
    println("3")
    rec_A = gen_B(fake_B)
    rec_B = gen_A(fake_A)
    
    ### Generator Losses ###
    # For domain A->B  #
    println("4")
    gen_B_loss = mean((fake_B_prob .- real_labels).^2)
    rec_B_loss = mean(abs.(X_B .- rec_B)) # Reconstruction loss for domain B
    
    # For domain B->A  #
    println("5")
    gen_A_loss = mean((fake_A_prob .- real_labels).^2)
    rec_A_loss = mean(abs.(X_A .- rec_A)) # Reconstrucion loss for domain A
    
    gen_loss = gen_A_loss + gen_B_loss + λ₁*rec_A_loss + λ₂*rec_B_loss # Total generator loss
    
    println("Forward propagate generators")
    # Optimise
    gs = Tracker.gradient(() -> gen_loss,params(params(gen_A)...,params(gen_B)...))
    update!(opt_gen,params(params(gen_A)...,params(gen_B)...),gs)
    println("Optimised generators")
    
    ### Discriminator Losses ###
    # For domain A #
    zero_grad!(dis_A)
    fake_A_prob = drop_first_two(dis_A(fake_A.data))
    dis_A_real_loss = mean((real_A_prob .- real_labels).^2)
    dis_A_fake_loss = mean((fake_A_prob .- fake_labels).^2)
    dis_A_loss = 0.5 * (dis_A_real_loss + dis_A_fake_loss)
    println("Forward propagate disA")
    gs = Tracker.gradient(() -> dis_A_loss,params(dis_A))
    update!(opt_disc_A,params(dis_A),gs)
    println("Optimised disA")
    
    # For domain B #
    zero_grad!(dis_B)
    fake_B_prob = drop_first_two(dis_B(fake_B.data))
    dis_B_real_loss = mean((real_B_prob .- real_labels).^2)
    dis_B_fake_loss = mean((fake_B_prob .- fake_labels).^2)
    dis_B_loss = 0.5 * (dis_B_real_loss + dis_B_fake_loss)
    println("Forward propagate disB")
    gs = Tracker.gradient(() -> dis_B_loss,params(dis_B))
    update!(opt_disc_B,params(dis_B),gs)
    println("Optimised disB")
    
    return gen_loss,dis_A_loss,dis_B_loss
end

println("Training...")
for epoch in 1:NUM_EPOCHS
    println("-----------Epoch : $epoch-----------")
    for i in 1:length(train_A)
        g_loss,dA_loss,dB_loss = train_step(train_A[i] |> gpu,train_B[i] |> gpu)
        if epoch % VERBOSE_FREQUENCY == 0
            println("Gen Loss : $g_loss")
            println("DisA Loss : $dA_loss")
            println("DisB Loss : $dB_loss")
        end
    end
end

Loaded Data
Loaded Models
Training...
-----------Epoch : 1-----------
1
A
A
A
A
2
A
A
A
A
3
A
A
A
A


OutOfMemoryError: OutOfMemoryError()

In [4]:
model = Discriminator() |> gpu
model_ = Discriminator() |> gpu
params(model)
params(model_)

Params([Float32[-0.438577 -0.191194 0.541819 -0.0620274; 0.353429 0.447337 0.277828 -0.228345; -0.0347384 -0.328329 -0.461335 0.258421; -0.496297 -0.496884 0.328832 -0.265421]

Float32[0.237065 0.209734 0.315213 -0.190872; 0.0787106 -0.537421 0.19522 -0.367137; -0.154398 0.137069 0.150036 0.420148; 0.388743 0.241512 -0.238598 0.490312]

Float32[-0.295243 -0.177881 -0.112692 0.223978; -0.145811 -0.393379 -0.114814 0.373665; 0.53101 0.0539193 -0.361691 0.239553; -0.198937 0.315052 -0.539774 -0.162018]

Float32[0.471885 -0.0194265 -0.479232 -0.389573; 0.252365 -0.285382 -0.198345 0.0343673; -0.0136015 -0.209871 -0.449582 -0.321796; -0.0965683 -0.471775 0.310154 0.330831]

Float32[0.357595 0.0452656 -0.114731 0.129791; -0.412887 0.0197597 0.414622 -0.0221095; 0.0514459 0.0498927 -0.172996 -0.331251; 0.397014 0.0473926 -0.503828 -0.497522]

Float32[-0.534552 0.397452 -0.190885 -0.301726; 0.553543 -0.0658325 0.145282 0.440122; 0.352974 -0.525716 0.412005 -0.402797; 0.308157 -0.279881 0.48703

In [37]:
out = drop_first_two(model(ones(256,256,3,2) |> gpu))
out_ = drop_first_two(model_(ones(256,256,3,2) |> gpu))
los = mean((out .+ out_ .- (ones(size(out)) |> gpu)).^2)
gs = Tracker.gradient(() -> los,params(params(model)...,params(model_)...))
println("DONE :)")

DONE :)


In [25]:
model = UNetConvBlock(3,16) |> gpu

Chain(Conv((3, 3), 3=>16), getfield(Main, Symbol("##43#45"))(), BatchNorm(16), getfield(Main, Symbol("##44#46"))(), getfield(Main, Symbol("##47#49"))(), Conv((3, 3), 16=>16), getfield(Main, Symbol("##43#45"))(), BatchNorm(16), getfield(Main, Symbol("##44#46"))(), getfield(Main, Symbol("##48#50"))())

In [26]:
model(ones(256,256,3,1) |> gpu)

Tracked 256×256×16×1 CuArray{Float32,4}:
[:, :, 1, 1] =
 -0.0255737  -0.0659975  -0.0728498    …  -0.0728498    -0.0675613    3.34834
 -0.0253359   1.86922     0.289874         0.289874     -0.0155184   10.5983 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698  …  -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698  …  -0.000237698  -0.00531767  13.6651 
 -0.0276155  -0.0110022  -0.000237698     -0.000237698  -0.00531767  13.6651 
 -0.0276

In [23]:
BatchNormWrap(16)(ones(256,256,16,1))

Tracked 256×256×16×1 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0 

In [4]:
# Hyperparameters
NUM_EPOCHS = 100
BATCH_SIZE = 1
dis_lr = 0.0001f0
gen_lr = 0.0001f0
λ₁ = 10.0 # Cycle loss weight for dommain A
λ₂ = 10.0 # Cycle loss weight for domain B
NUM_EXAMPLES = 2 # Temporary for experimentation
VERBOSE_FREQUENCY = 2 # Verbose output after every 2 epochs

# Data Loading
dataA = load_dataset("../data/trainA/",256)[:,:,:,1:NUM_EXAMPLES] |> gpu
dataB = load_dataset("../data/trainB/",256)[:,:,:,1:NUM_EXAMPLES] |> gpu
mb_idxs = partition(shuffle!(collect(1:size(dataA)[end])), BATCH_SIZE)
train_A = [make_minibatch(dataA, i) for i in mb_idxs]
train_B = [make_minibatch(dataB, i) for i in mb_idxs]
println("Loaded Data")

# Define Optimizers
opt_gen = ADAM(gen_lr,(0.5,0.999))
opt_disc_A = ADAM(dis_lr,(0.5,0.999))
opt_disc_B = ADAM(dis_lr,(0.5,0.999))

# Define models
gen_A = UNet() |> gpu # Generator For A->B
gen_B = UNet() |> gpu # Generator For B->A
dis_A = Discriminator() |> gpu # Discriminator For Domain A
dis_B = Discriminator() |> gpu # Discriminator For Domain B
println("Loaded Models")

Loaded Data
Loaded Models


In [6]:
gen_A(ones(256,256,3,1) |> gpu)

A
A
A
A


Tracked 256×256×3×1 CuArray{Float32,4}:
[:, :, 1, 1] =
 0.0644297  0.99233   1.0       0.998941  …  0.999827   0.995585   0.999885
 0.999999   0.999758  1.0       0.997472     1.0        0.640084   0.999997
 0.999976   0.999951  0.999998  1.0          0.999876   0.888759   0.998224
 0.999996   1.0       0.999998  0.9622       0.999934   0.994403   0.998673
 0.983807   0.999996  0.999585  0.999213     1.0        0.982215   0.998673
 0.999887   1.0       1.0       0.999988  …  0.999997   0.999359   0.99996 
 0.998634   0.999875  0.998912  1.0          0.999999   0.998037   0.999677
 0.999994   0.999999  1.0       0.999801     0.999994   0.691239   0.999995
 0.967285   0.999613  0.996412  0.999786     1.0        0.99975    0.999367
 0.999945   0.999992  0.999999  0.999503     1.0        0.996869   0.999995
 0.999095   0.999949  0.999965  0.999884  …  0.999999   0.999945   0.999846
 0.999997   0.999996  1.0       0.980448     0.999998   0.713162   0.999979
 0.988247   0.999828  0.945386  0

In [26]:
"""
image_pool.jl
"""

mutable struct ImagePool
    pool_size
    pool
end

ImagePool(pool_size::Int) = ImagePool(pool_size,[])

function query(ip::ImagePool,images)
    if ip.pool_size > length(ip.pool)
        push!(ip.pool,images) 
        return ip.pool
    else
        pool_copy = ip.pool
        ip.pool[1:end-1] = pool_copy[2:end]
        ip.pool[end] = images
        
        p = rand()
        if p > 0.5
           return images 
        else
            return ip.pool[1]
        end
    end
end

query (generic function with 1 method)

In [27]:
ip = ImagePool(3)

ImagePool(3, Any[])

In [28]:
query(ip,ones(1,3,256,256))
query(ip,ones(1,3,256,256))
query(ip,ones(1,3,256,256))
query(ip,zeros(1,3,256,256))
query(ip,zeros(1,3,256,256))

1×3×256×256 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  0.0

...

[:, :, 254, 1] =
 0.0  0.0  0.0

[:, :, 255, 1] =
 0.0  0.0  0.0

[:, :, 256, 1] =
 0.0  0.0  0.0

[:, :, 1, 2] =
 0.0  0.0  0.0

[:, :, 2, 2] =
 0.0  0.0  0.0

[:, :, 3, 2] =
 0.0  0.0  0.0

...

[:, :, 254, 2] =
 0.0  0.0  0.0

[:, :, 255, 2] =
 0.0  0.0  0.0

[:, :, 256, 2] =
 0.0  0.0  0.0

[:, :, 1, 3] =
 0.0  0.0  0.0

[:, :, 2, 3] =
 0.0  0.0  0.0

[:, :, 3, 3] =
 0.0  0.0  0.0

...

[:, :, 254, 3] =
 0.0  0.0  0.0

[:, :, 255, 3] =
 0.0  0.0  0.0

[:, :, 256, 3] =
 0.0  0.0  0.0

...

[:, :, 1, 254] =
 0.0  0.0  0.0

[:, :, 2, 254] =
 0.0  0.0  0.0

[:, :, 3, 254] =
 0.0  0.0  0.0

...

[:, :, 254, 254] =
 0.0  0.0  0.0

[:, :, 255, 254] =
 0.0  0.0  0.0

[:, :, 256, 254] =
 0.0  0.0  0.0

[:, :, 1, 255] =
 0.0  0.0  0.0

[:, :, 2, 255] =
 0.0  0.0  0.0

[:, :, 3, 255] =
 0.0  0.0  0.0

...

[:, :, 254, 255] =
 0.0  0.0  0.0

[:, :, 255, 255] =
 0.0  0

In [25]:
ip.pool

3-element Array{Any,1}:
 [1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

...

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

...

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

...

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

...

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

...

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

...

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]

...

[1.0 1.0 1.0]

[1.0 1.0 1.0]

[1.0 1.0 1.0]
 [0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

...

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

...

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

...

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

...

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

...

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]

[0.0 0.0 0.0]



In [6]:
a

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 1.0

In [29]:
using Random

In [35]:
rand()

0.7842050673225829